In [ ]:
import sys
sys.path.append("../..")

import os
import pickle
import random


import pandas as pd
from pandas import DataFrame

from app.components.extract_data.extract_data import getDfPerson, getDfConsumo, getDictV9001toGroupPt, getDictV9001toGroupEn
from app.components.simple_types import Nutrition, State
from app.components.search.papa_search import SearchResult, papaSingleSeach, fitness

Loading basic data
Loading basic dictionary
Finish!


In [2]:
datasetPath = os.getcwd() + "/../../datasets/"
datasetPicklePath = os.getcwd() + "/../../datasets/pickle/"

In [3]:
dfConsumo = getDfConsumo()
dfPerson = getDfPerson()

In [4]:
len(dfPerson["ESTRATO_POF"].unique())

575

In [ ]:
class PersonInformation:
    def __init__(self, person:str):
        self.id:str = person
        # Dieta inicial da pessoa
        self.initialDiet:State = State.getStateByPersonId(person)        
        # Dieta inicial do passo atual
        self.actualDiet:State = State.getStateByPersonId(person) 
        # Nutriçao recomendada para a pessoa
        self.targetNutrition= Nutrition.idealNutritionByPersonId(person) 

In [6]:
class DietAgent:
    def __init__(self, person:str):
        self.personInformation:PersonInformation = PersonInformation(person)
        self.optimizedDiets:list[State] = [State.getStateByPersonId(person)]

In [ ]:
class DietAllyEnemy:
    def __init__(self, person:str):
        self.enemy:int = -1
        self.ally:int = -1
        self.bestDiet:State = State.getStateByPersonId(person) 
        self.bestDietFitness:float = None
    

In [ ]:
class HealthAgent:
    def __init__(self):
        self.personDietAllyEnemmy:dict[str, DietAllyEnemy] = {}
        self.dietAgents:dict[str, DietAgent] = {}
        
    def add_person(self, person:str): 
        self.personDietAllyEnemmy[person] = DietAllyEnemy(person)
        self.dietAgents[person] = DietAgent(person)
        
    def assign_social(self):
        allies = list(self.dietAgents.keys())
        enemies = list(self.dietAgents.keys())
        random.shuffle( allies )
        random.shuffle( enemies )
        
        people = list(self.dieatAgents.keys())
        for id, person in enumerate(people):
            self.personDietAllyEnemmy[person].ally = allies[id]
            self.personDietAllyEnemmy[person].enemy = enemies[id]
            
    def updateBest(self, person:str, candidateBest:State, nutritionFactor:float=1.2, distanceFactor:float=0.1):
        self.personDietAllyEnemmy[person]
        
        if self.personDietAllyEnemmy[person].bestDietFitness == None:
            self.personDietAllyEnemmy[person].bestDietFitness = fitness(
                state=self.personDietAllyEnemmy[person].bestDiet,
                initialState=self.dietAgents[person].personInformation.actualDiet,
                targetNutrition=self.dietAgents[person].personInformation.targetNutrition,
                nutritionFactor=nutritionFactor,
                distanceFactor=distanceFactor, 
            )
        
        candidateFitness = fitness(
            state=candidateBest,
            initialState=self.dietAgents[person].personInformation.actualDiet,
            targetNutrition=self.dietAgents[person].personInformation.targetNutrition,
            nutritionFactor=nutritionFactor,
            distanceFactor=distanceFactor, 
        )
        
        if( self.personDietAllyEnemmy[person].bestDietFitness > candidateFitness ):
            self.personDietAllyEnemmy[person].bestDietFitness = candidateFitness
            self.personDietAllyEnemmy[person].bestDiet = candidateBest
            

In [9]:
personID = "110000016#7#1#5"

In [10]:
healthAgent = HealthAgent()
dietAgent = DietAgent(personID)

In [11]:
healthAgent.add_person(personID)

In [12]:
#Find people 
dfPerson = getDfPerson()
stratos = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["ESTRATO_POF"].to_list()
quantity = 10
male25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
male60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="male") ]["PESSOA"].to_list()[:quantity]
female25 = dfPerson[ (dfPerson["AGE"]==25) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]
female60 = dfPerson[ (dfPerson["AGE"]==60) & (dfPerson["GENDER"]=="female") ]["PESSOA"].to_list()[:quantity]

In [ ]:
def multiAgentSearch(
        people:list[str], 
        resultFileName:str, 
        maxSteps:int = 10,
        nutritionFactor:float = 1.2,
        distanceFactor:float = 0.1, 
        verbose:bool=False) -> list[list[SearchResult]]:
    try:
        with open("./output/" + resultFileName, "rb") as file:
            return pickle.load(file)
    except:
        
        healthAgent = HealthAgent()
        for person in people:
            healthAgent.add_person(person)
        healthAgent.assign_social()
        
        results = []
        phases = ["corward", "hero"]
        
        for interation in range(3):
            # RUN multiAgentSearch
            for step in range(maxSteps):
                # operator:function = None
                if phases[step%2] == "corward":
                #     # TODO: Fazer o operador covarde
                #     operator = cowardOperator
                    pass
                elif phases[step%2] == "hero":
                #     # TODO: Fazer o operador heroi
                #     operator = heroOperator
                    pass
                                
                for agentId, agent in enumerate(healthAgent.dieatAgents):
                    person = agent.personInformation.id
                    
                    result = papaSingleSeach(
                                personID=person, 
                                verbose=False, 
                                unit=10, 
                                max_unit=2, 
                                max_population_set=20, 
                                max_population_selected=10, 
                                expansion_set=20, 
                                expansion_select=5, 
                                max_steps=1, 
                                nutritionFitness=Nutrition.absDifferenceNegativePenalty, 
                                nutritionFactor=nutritionFactor,
                                distanceFitness=State.squareDifference,
                                distanceFactor = distanceFactor,
                                preserve_best=True,
                                crossover=0.1, 
                                initialState=agent.personInformation.actualDiet,
                                initialPopulation=agent.optimizedDiets,
                            )

                    healthAgent.updateBest(
                        person=person,
                        candidateBest=result.finalMeal,
                        )
                    
                    roundBestFitness = fitness(
                        state=result.finalMeal,
                        initialState=agent.personInformation.actualDiet,
                        targetNutrition= agent.personInformation.targetNutrition,
                        nutritionFactor=nutritionFactor,
                        distanceFactor=distanceFactor,
                    )
                    
                    healthAgent.personDietAllyEnemmy[person].diet
                    if ( roundBestFitness < agent.personInformation.id)
                        
                        
                    break
                break
            break
                    
                
            
            
            # Build Results
            for i, person in enumerate(people):
                results[interation].append(
                            SearchResult(
                                [person],
                                healthAgent.dieatAgents[i].personInformation.initialDiet,
                                healthAgent.personDietAllyEnemmy[i].diet,
                            ))
        
        # with open("./output/" + resultFileName, "wb") as file:
        #     pickle.dump(results, file) 
        
        return results
            

In [15]:
print(male25)
results:list[list[SearchResult]] = []
results = multiAgentSearch(
                people=male25, 
                resultFileName="male25_100_agent.pickle", 
                maxSteps=3, 
                verbose=True
            )

print(results)

['110004584#15#1#1', '110010188#3#1#4', '110012387#6#1#3', '110021380#9#1#1', '110021533#5#1#1', '120001153#9#1#3', '120002090#16#1#4', '120005005#10#1#1', '120005365#7#1#3', '120005651#8#1#4']
Before: [State()]
initialPopulation: [State(), State(), State(), State(), State(), State(), State(), State(), State(), State(), State()]
After: [State(), State(), State(), State(), State(), State(), State(), State(), State(), State(), State()]
[]


In [21]:
d = {
    2:4,
    3:9,
    4:16
}

print(list(d.keys()))

for a, b in d.items():
    print(a, b)

[2, 3, 4]
2 4
3 9
4 16
